# Driving Exam Auto Tagging
Direct tagging with a VLM

## A. Load Question Data

## 0. Set up environment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys

SRC_PATH = "/content/drive/Othercomputers/My MacBook Pro/Projects/Drivetest App/2 - NLP Tag Creation/drivetest_tag_extraction/src"
sys.path.append(SRC_PATH)
# SRC_PATH = ""

In [3]:
!pip3 install qwen-vl-utils
!pip3 install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 25.8 MB/s eta 0:00:00


In [4]:
MY_DEVICE = "cuda"

### 1. Import the scraped question bank

In [5]:
import os

from qb.question_bank import QuestionBank
from data_storage.database.local_json_db import LocalJsonDB
from data_cleaning.data_formatter import DataFormatter
from data_cleaning.img_reshaper import ImgSquarer

In [6]:
RAW_DATA_DIR = os.path.join(SRC_PATH, "data_storage/database/json_db")
FORMATTED_DATA_DIR = os.path.join(SRC_PATH, "data_storage/formatted_database")

def load_data(make_new_data=False) -> QuestionBank:
    """ Load the question bank from the formatted data directory """
    if not os.path.exists(FORMATTED_DATA_DIR) or not os.listdir(FORMATTED_DATA_DIR) or make_new_data:
        os.makedirs(FORMATTED_DATA_DIR, exist_ok=True)
        raw_db = LocalJsonDB(os.path.join(RAW_DATA_DIR, "data.json"),
                             os.path.join(RAW_DATA_DIR, "images"))
        data_formatter = DataFormatter(image_reshaper=(ImgSquarer(256)))
        qb = data_formatter.format_data(raw_db, FORMATTED_DATA_DIR)
    else:
        data = LocalJsonDB(os.path.join(FORMATTED_DATA_DIR, "data.json"),
                         os.path.join(FORMATTED_DATA_DIR, "images"))
        qb = data.load()
    return qb
qb = load_data(make_new_data=False)
print(len(qb.get_qid_list()))

2836


## B. Auto Tagging with VLM

In [7]:
from typing import Dict, List, Any
from qb.question import Question

In [8]:
def get_prompt() -> str:
    prompt = """你是一位驾考科目一领域的知识分类专家。你的核心任务是分步思考，综合分析给出的【章节】、【题目】、【选项】和【答案】，为这道题提炼并归纳出用于分类的高阶`tags`（标签）和用于搜索的精细`keywords`（关键词）。

---
### 思考与输出格式

你必须严格遵循以下两步进行输出：

**第一步：思考过程**
你需要按下面的步骤展示你的分析逻辑：
* **步骤1：情境概括。** 用一句话概括这道题的完整场景和核心问题。
* **步骤2：关键词提取。** 从题目所有信息中，提取最核心、最具体的关键词。此步骤的输出将直接作为最终的`keywords`。
* **步骤3：标签归纳与升维。** 针对上一步提取的关键词，思考它们各自所属的更高阶的“概念”或“分类”。例如，从“70周岁”可以升维至“高龄驾驶员”。
* **步骤4：最终决策。** 从上一步归纳出的所有高阶概念中，选择2-6个最相关、最具有代表性的作为最终的`tags`。

**第二步：最终输出**
在完成思考过程后，另起一行，以一个包含`tags`和`keywords`两个键的JSON对象格式输出。
* `tags`字段中放入步骤4决策的最终标签数组。
* `keywords`字段中放入步骤2提取的（不重复的）关键要素数组。

---
### 示范

**输入 1:**
{
  "章节": "1: 道路交通安全法律、法规和规章",
  "题目": "年满70周岁以上的机动车驾驶人发生责任交通事故造成人员重伤或者死亡的，应当在本记分周期结束后三十日内到公安机关交通管理部门接受审验。",
  "选项": {"A": "对", "B": "错"},
  "答案": "A"
}

**输出 1:**
思考过程:
步骤1：情境概括。本题考察的是关于70岁以上驾驶员在发生严重责任事故后，必须在规定期限内接受驾驶证审验的法规。
步骤2：关键词提取。70周岁、责任交通事故、重伤死亡、审验、三十日内、记分周期。
步骤3：标签归纳与升维。“70周岁” -\> 高龄驾驶员；“责任交通事故”、“重伤死亡” -\> 交通事故责任；“审验”、“记分周期”、“三十日内” -\> 驾驶证审验；题目来源的章节 -\> 法律法规。
步骤4：最终决策。选择“驾驶证审验”、“高龄驾驶员”、“交通事故责任”、“法律法规”作为最核心的标签。

最终输出:
{
  "tags": ["驾驶证审验", "高龄驾驶员", "交通事故责任", "法律法规"],
  "keywords": ["70周岁", "责任交通事故", "重伤死亡", "审验", "三十日内", "记分周期"]
}

**输入 2:**
{
  "章节": "2: 交通信号",
  "题目": "浓雾天气能见度低，开启远光灯会提高能见度。",
  "选项": {"A": "对", "B": "错"},
  "答案": "B"
}

**输出 2:**
思考过程:
步骤1：情境概括。本题考察的是在浓雾天气下，错误使用远光灯反而会降低能见度的安全知识。
步骤2：关键词提取。浓雾、能见度低、远光灯、视野、错误操作。
步骤3：标签归纳与升维。“浓雾” -\> 恶劣天气驾驶；“远光灯” -\> 灯光使用；“能见度低”、“视野” -\> 安全视野。
步骤4：最终决策。选择“恶劣天气驾驶”、“灯光使用”、“安全视野”作为最核心的标签。

最终输出:
{
  "tags": ["恶劣天气驾驶", "灯光使用", "安全视野"],
  "keywords": ["浓雾", "能见度低", "远光灯", "视野", "漫反射"]
}

**输入 3:**
{
  "章节": "3: 安全行车、文明驾驶基础知识",
  "题目": "参加公安机关交通管理部门组织的道路交通安全法律、法规和相关知识网上学习三日内累计满三十分钟且考试合格的，一次扣减1分交通违法行为记分。",
  "选项": {"A": "对", "B": "错"},
  "答案": "A"
}

**输出 3:**
思考过程:
步骤1：情境概括。本题考察的是关于“学法减分”政策的具体执行标准，包括学习时长和减免分数。
步骤2：关键词提取。网上学习、考试合格、扣减1分、记分、三日内、三十分钟。
步骤3：标签归纳与升维。“网上学习”、“扣减1分” -\> 学法减分；“记分” -\> 记分管理；题目来源的章节 -\> 法律法规。
步骤4：最终决策。选择“学法减分”、“记分管理”、“法律法规”作为最核心的标签。

最终输出:
{
  "tags": ["学法减分", "记分管理", "法律法规"],
  "keywords": ["网上学习", "考试合格", "扣减1分", "记分", "三日内", "三十分钟"]
}

**输入 4:**
{
  "章节": "4: 机动车驾驶操作相关基础知识",
  "题目": "机动车仪表板上如图所示这个符号表示什么？",
  "选项": {"A": "后雾灯开关", "B": "车灯总开关", "C": "近光灯开关", "D": "远光灯开关"},
  "答案": "B"
}

**输出 4:**
思考过程:
步骤1：情境概括。本题考察的是对仪表板上一个特定符号（车灯总开关）的辨认能力。
步骤2：关键词提取。仪表板、符号、车灯总开关、后雾灯、近光灯、远光灯。
步骤3：标签归纳与升维。“仪表板”、“符号” -\> 仪表与指示灯；“车灯总开关”、“后雾灯开关”等选项 -\> 灯光开关、车辆功能按键。
步骤4：最终决策。选择“仪表与指示灯”、“车辆功能按键”、“灯光开关”作为最核心的标签。

最终输出:
{
  "tags": ["仪表与指示灯", "车辆功能按键", "灯光开关"],
  "keywords": ["仪表板", "符号", "车灯总开关", "后雾灯", "近光灯", "远光灯"]
}

**输入 5:**
{
  "章节": "5: 货车专用试题",
  "题目": "安全头枕在发生追尾事故时，能有效保护驾驶人的什么部位？",
  "选项": {"A": "头部", "B": "胸部", "C": "腰部", "D": "颈部"},
  "答案": "D"
}

**输出 5:**
思考过程:
步骤1：情境概括。本题考察的是车辆被动安全装置“安全头枕”在追尾事故中对驾驶员颈部的保护功能。
步骤2：关键词提取。安全头枕、追尾事故、保护、颈部、货车。
步骤3：标签归纳与升维。“安全头枕” -\> 车辆安全装置、被动安全；“追尾事故” -\> 追尾事故防护；“颈部” -\> 驾驶员防护。
步骤4：最终决策。选择“车辆安全装置”、“被动安全”、“追尾事故防护”作为最核心的标签。

最终输出:
{
  "tags": ["车辆安全装置", "被动安全", "追尾事故防护"],
  "keywords": ["安全头枕", "追尾事故", "保护", "颈部", "货车"]
}

**输入 6:**
{
  "章节": "6: 客车专用试题",
  "题目": "再次饮酒后驾驶机动车的，不得申请大型客车准驾车型。",
  "选项": {"A": "对", "B": "错"},
  "答案": "A"
}

**输出 6:**
思考过程:
步骤1：情境概括。本题考察的是因有“再次酒驾”这一严重违法行为历史，导致驾驶人被限制申请大型客车准驾车型的法律规定。
步骤2：关键词提取。再次饮酒、驾驶、不得申请、大型客车、准驾车型。
步骤3：标签归纳与升维。“再次饮酒” -\> 饮酒驾驶、违法行为处罚；“不得申请”、“准驾车型” -\> 驾驶证申领；“大型客车” -\> 客车规定。
步骤4：最终决策。选择“驾驶证申领”、“饮酒驾驶”、“客车规定”、“违法行为处罚”作为最核心的标签。

最终输出:
{
  "tags": ["驾驶证申领", "饮酒驾驶", "客车规定", "违法行为处罚"],
  "keywords": ["再次饮酒", "驾驶", "不得申请", "大型客车", "准驾车型"]
}

**输入 7:**
{
  "章节": "7: 摩托车专用试题",
  "题目": "驾驶机动车在道路上向右变更车道可以不使用转向灯。",
  "选项": {"A": "对", "B": "错"},
  "答案": "B"
}

**输出 7:**
思考过程:
步骤1：情境概括。本题考察的是变更车道前必须使用转向灯这一基本安全驾驶操作规范。
步骤2：关键词提取。变更车道、转向灯、向右、摩托车、不使用。
步骤3：标签归-维。“变更车道” -\> 变更车道、安全驾驶操作；“转向灯” -\> 转向灯使用；该行为规范 -\> 道路通行规则。
步骤4：最终决策。选择“安全驾驶操作”、“变更车道”、“转向灯使用”、“道路通行规则”作为最核心的标签。

最终输出:
{
  "tags": ["安全驾驶操作", "变更车道", "转向灯使用", "道路通行规则"],
  "keywords": ["变更车道", "转向灯", "向右", "摩托车", "示意"]
}
"""
    return prompt

In [9]:
def format_question(question: Question) -> str:
    """ Format the question into a json string """
    dict_question = {"章节": f"{question.get_chapter()[0]}: {question.get_chapter()[1]}",
                     "题目": question.get_question(),
                     "选项": {},
                     "答案": ""}

    answer_choices = list(question.get_answers())
    answer_choices.sort()
    for i in range(0, len(answer_choices)):
        letter_code = chr(ord('A') + i)
        dict_question["选项"][letter_code] = answer_choices[i]
        if answer_choices[i] == question.get_correct_answer():
            dict_question["答案"] = letter_code
    return str(dict_question)

In [10]:
def make_content(question: Question) -> List[Dict[str, Any]]:
    if question.get_img_path() is not None:
        return [
            {"type": "image",
             "image": question.get_img_path()},
            {"type": "text",
             "text": format_question(question)},
        ]
    else:
        return [
            {"type": "text",
             "text": format_question(question)}
        ]

In [11]:
def make_messages(qb: QuestionBank) -> List[List[Dict[str, Any]]]:
    input_lst = []
    for qid in qb.get_qid_list():
        question = qb.get_question(qid)
        input_lst.append([{"role": "system",
                           "content": get_prompt()},
                          {"role": "user",
                           "content": make_content(question)}])
    return input_lst

In [12]:
from transformers import AutoProcessor, AutoModelForVision2Seq
import torch
from qwen_vl_utils import process_vision_info

In [35]:
MODEL_NAME = "moot20/Qwen2.5-VL-3B-Instruct-MLX-4bits"
model_save_path = os.path.join(SRC_PATH, "data_storage/model_dir/", MODEL_NAME.split("/")[-1])

def make_model_and_processor(model_name: str,
                             model_save_path: str,
                             make_new:bool = False,
                             device:str = MY_DEVICE):
  if make_new:
    model = AutoModelForVision2Seq.from_pretrained(MODEL_NAME,
                                                   torch_dtype=torch.float16,
                                                   device_map=device)

    min_pixels = 256 * 28 * 28
    max_pixels = 257 * 28 * 28
    processor = AutoProcessor.from_pretrained(
        model_name,
        min_pixels=min_pixels,
        max_pixels=max_pixels,
        use_fast=True)
    model.save_pretrained(model_save_path)
    processor.save_pretrained(model_save_path)
  else:
    model = AutoModelForVision2Seq.from_pretrained(model_save_path,
                                                   torch_dtype=torch.float16,
                                                   device_map=device)
    processor = AutoProcessor.from_pretrained(model_save_path)
  return model, processor
model, processor = make_model_and_processor(MODEL_NAME, model_save_path, make_new=True)

ValueError: You current version of `autoawq` does not support module quantization skipping, please upgrade `autoawq` package to at least 0.1.8.

In [ ]:
from datetime import datetime
import pandas as pd
from pandas import DataFrame

In [13]:
def make_inputs(messages, processor, device=MY_DEVICE) -> Dict[str, torch.Tensor]:
    """
    Prepare the inputs for the model from the messages.
    """
    texts = []
    for msg in messages:
        texts.append(processor.apply_chat_template(msg, tokenize=False, add_generation_prompt=True))
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=texts,
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(device)
    return inputs

In [ ]:
def result_to_df(qid_lst: List[str], messages: List[List[Dict[str, Any]]], result: List[str]) -> DataFrame:
    """ Convert the result into a dataframe """
    data = {
        "qid": qid_lst,
        "question": [],
        "tags": [],
        "keywords":[]
    }
    for msg in messages:
        question_text = msg[-1]["content"][-1]["text"]
        data["question"].append(question_text)
    for output in result:
        tag_start = output.rfind("tags: [")
        tag_end = output[tag_start:].find("]")
        keywords_start = output.rfind("keywords: [")
        keywords_end = output[keywords_start:].find("]")
        tags = output[tag_start + 7: tag_end + tag_start].strip().split(", ")
        keywords = output[keywords_start + 11: keywords_end + keywords_start].strip().split(", ")
        data["tags"].append(tags)
        data["keywords"].append(keywords)
    return DataFrame(data)

In [ ]:
def save_labeling_data(qid_lst: List[str], messages: List[List[Dict[str, Any]]], results: List[str], tagging_results_dir) -> None:
    """ Save the tagging results to a CSV file """
    if not os.path.exists(tagging_results_dir):
        os.makedirs(tagging_results_dir)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    tagging_results_file = os.path.join(tagging_results_dir, f"tagging_results_{timestamp}.csv")
    tagging_results = result_to_df(qid_lst, messages, results)
    tagging_results.to_csv(tagging_results_file, index=False)

In [ ]:
def load_labeling_data(directory: str) -> DataFrame:
    """ Load the tagging results from the latest CSV file """
    files = os.listdir(directory)
    if not files:
        return DataFrame()
    latest_file = max(files, key=lambda x: os.path.getctime(os.path.join(directory, x)))
    return pd.read_csv(os.path.join(directory, latest_file))

In [ ]:
def make_batches(qb: QuestionBank, messages: List[str], batch_size: int) -> (List[List[str]], List[List[str]]):
    """ Split the messages into batches """
    qid_batches = []
    qid_list = qb.get_qid_list()

    msg_batches = []
    for i in range(0, len(messages), batch_size):
        qid_batch = qid_list[i:i + batch_size]
        qid_batches.append(qid_batch)

        batch = messages[i:i + batch_size]
        msg_batches.append(batch)
    return qid_batches, msg_batches

In [ ]:
def generate_labels(messages: List[str], model, processor) -> List[str]:
    inputs = make_inputs(messages, processor)
    generated_ids = model.generate(**inputs, max_new_tokens=512)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    results = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    return results

In [ ]:
TAGGING_RESULTS_DIR = os.path.join(SRC_PATH, "data_storage/tagging_results")
messages = make_messages(qb)
qid_batches, msg_batches = make_batches(qb, messages, batch_size=10)
print(len(qid_batches), len(msg_batches))

In [ ]:
for i in range(len(msg_batches)):
    print(f"Processing batch {i + 1}/{len(msg_batches)}")
    start_time = datetime.now()
    inputs = make_inputs(msg_batches[i], processor)
    generated_ids = model.generate(**inputs, max_new_tokens=1024)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    results = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    save_labeling_data(qid_batches[i], msg_batches[i], results, TAGGING_RESULTS_DIR)
    end_time = datetime.now()
    print(f"Batch {i + 1} processed in {end_time - start_time} seconds")

In [ ]:
result_df = load_labeling_data(TAGGING_RESULTS_DIR)
result_df.head()